In [1]:
from pandas import read_csv
from scipy.stats import geom, norm, multinomial, binom
import matplotlib.pyplot as plt 
import numpy as np

In [2]:
def theta_to_data(N, pi, alpha, lambda_, mu_2, sigma):
    X = [0 for _ in range(int(pi[0] * N * alpha))] # generating zeros
    X_l = geom.rvs(1/lambda_, size=(int(pi[0] * N * (1-alpha)))) # generating observations following geometric 
    X.extend(X_l)

    X_g = norm.rvs(mu_2, sigma[0], size =(int(pi[1] * N))) # generating observations following normal
    X_g = [round(g) for g in X_g] # converting to integers
    X.extend(X_g)

    X_g2 = norm.rvs(2*mu_2, sigma[1], size =(int(pi[2] * N))) # generating observations following normal
    X_g2 = [round(g) for g in X_g2] # converting to integers
    X.extend(X_g2)

    X = np.array(X)
    return X

# draw from the multinomial distribution using the pi vector
# round and then redraw to drop the zeroes, count zeros and then redraw
# sanity check, if any values are less than zero, then break

In [3]:
X = theta_to_data(1000, [1/3, 1/3, 1/3], 0.5, 3, 5, [3, 3])
assert(0 == sum(X < 0))
print(X)

AssertionError: 

In [2]:
def theta_to_data(N, pi, alpha, lambda_, mu_2, sigma):

    draws = multinomial.rvs(n = N, p = pi)
    draws_alpha = multinomial.rvs(n = draws[0], p = [alpha, 1 - alpha])

    X = [0 for _ in range(draws_alpha[0])]
    X_l = geom.rvs(1/lambda_, size = draws_alpha[1])
    X_g = norm.rvs(mu_2, sigma[0], size = draws[1])
    X_g2 = norm.rvs(2 * mu_2, sigma[1], size = draws[2])

    while sum(X_g < 0) > 0:
        X_g_new = norm.rvs(mu_2, sigma[0], size = sum(X_g < 0))
        X_g = [x for x in X_g if x >= 0]
        X_g = np.concatenate((X_g, X_g_new))

    while sum(X_g2 < 0) > 0:
        X_g2_new = norm.rvs(2 * mu_2, sigma[1], size = sum(X_g2 < 0))
        X_g2 = [x for x in X_g2 if x >= 0]
        X_g2 = np.concatenate((X_g2, X_g2_new))

    X = np.concatenate((X,X_l))
    X = np.concatenate((X,X_g))
    X = np.concatenate((X,X_g2))
    X = np.round(X)

    return X

In [3]:
X = theta_to_data(1000, [1/3, 1/3, 1/3], 0.5, 3, 20, [3, 3])
assert(0 == sum(X < 0))
print(X)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  6.
  4.  3.  2.  3.  1.  4.  1.  1.  1.  3.  3.  1.  5.  1.  2.  1.  2.  5.
 15.  2.  8.  7.  2.  2.  8.  1.  5.  4.  2.  1.  1.  4.  1.  5.  4. 13.
  4.  2.  8.  1.  2.  5.  5.  2.  5.  2.  1. 17.  2.  4.  1.  2.  2.  1.
  1.  1.  1.  1.  1.  1.  2.  4.  1.  5.  2.  3.  8